# Project 02: One ligand against several targets

This notebook shows how plipify can be used to compare the binding modes of a small molecule towards multiple protein. This can be especially interesting for protein families with selectivity issues, for example kinases.

**Workflow:**  
01. Retrieve data from KLIFS  
02. Analyze data with plipify  
03. Generate publication ready figures  

## 01. Retrieve data from KLIFS

In [1]:
from opencadd.databases.klifs import setup_remote

INFO:opencadd.databases.klifs.api:If you want to see an non-truncated version of the DataFrames in this module, use `pd.set_option('display.max_columns', 50)` in your notebook.


In [2]:
# Set up remote session
remote = setup_remote()

INFO:opencadd.databases.klifs.api:Set up remote session...
INFO:opencadd.databases.klifs.api:Remote session is ready!


In [3]:
# retrieve kinase structures
kinase_df = remote.structures.all_structures()

In [4]:
# remove structures without orthosteric ligand
kinase_df = kinase_df[kinase_df["ligand.expo_id"] != "-"]

In [5]:
# sort by quality
kinase_df = kinase_df.sort_values(by=["structure.qualityscore", "structure.resolution"],ascending=[False, True])

In [6]:
# find unique kinase ligand pairs
kinase_ligand_pairs_df = kinase_df.groupby(["kinase.klifs_id", "ligand.expo_id"]).head(1)

In [7]:
# find ligands binding to many kinases
kinase_ligand_pairs_df.groupby("ligand.expo_id").size().sort_values(ascending=False).head(10)

ligand.expo_id
ANP    76
ADP    50
STU    40
ATP    31
ACP    19
AGS    17
1N1    14
AMP    10
STI    10
DB8    10
dtype: int64

## 02. Analyze data with plipify

TBD

## 03. Generate publication ready figures

TBD